La clase AdjacentVertex representa una tupla donde el primer elemento es un vértice y el segundo el peso asociado. 



# Problema 4

Este notebook contiene una solución explicada para el siguiente problema:

En la clase Graph2 hija de la clase Graph
(implementación de un grafo basado en diccionario), implementa un método,
minimum_path, que reciba dos vértices, start y end, y devuelva una lista que
contenga el camino mínimo de start a end, y la distancia del camino. Si el
camino no existe, deberá devolver una lista vacía y distancia infinito.

In [12]:
class AdjacentVertex:
    """ This class allows us to represent a tuple
    with an adjacent vertex
    and the weight associated (by default None, for non-unweighted graphs)"""
    def __init__(self, vertex: object, weight: int = 1) -> None:
        self.vertex = vertex
        self.weight = weight

    def __eq_(self, other: 'AdjacentVertex') -> bool:
        if other is None: 
            return False
        return self.vertex == other.vertex and self.weight == other.weight 
        
    def __str__(self) -> str:
        """ returns the tuple (vertex, weight)"""
        if self.weight is not None:
            return '(' + str(self.vertex) + ',' + str(self.weight) + ')'
        else:
            return str(self.vertex)

In [13]:

class Graph:
    def __init__(self, vertices: list, directed: bool = True) -> None:
        """ We use a dictionary to represent the graph
        the dictionary's keys are the vertices
        The value associated for a given key will be the list of their neighbours.
        Initially, the list of neighbours is empty"""
        self._vertices = {}
        for v in vertices:
            self._vertices[v] = []
        self._directed = directed

    def add_vertex(self, vertex: str) -> None:
        if vertex in self._vertices.keys():
            print(vertex, ' already exists!')
            return
        self._vertices[vertex] = []

    def add_edge(self, start: object, end: object, weight: int = 1) -> None:
        if start not in self._vertices.keys():
            print(start, ' does not exist!')
            return
        if end not in self._vertices.keys():
            print(end, ' does not exist!')
            return

        # adds to the end of the list of neighbours for start
        self._vertices[start].append(AdjacentVertex(end, weight))

        if not self._directed:
            # adds to the end of the list of neighbors for end
            self._vertices[end].append(AdjacentVertex(start, weight))

    def contain_edge(self, start: object, end: object) -> int:
        """ checks if the edge (start, end) exits. It does
        not exist return 0, eoc returns its weight or 1 (for unweighted graphs)"""
        if start not in self._vertices.keys():
            print(start, ' does not exist!')
            return 0
        if end not in self._vertices.keys():
            print(end, ' does not exist!')
            return 0

        # we search the AdjacentVertex whose v is equal to end

        for adj in self._vertices[start]:
            if adj.vertex == end:
                return adj.weight

        return 0  # does not exist

    def remove_edge(self, start: object, end: object):
        """ removes the edge (start, end)"""
        if start not in self._vertices.keys():
            print(start, ' does not exist!')
            return
        if end not in self._vertices.keys():
            print(end, ' does not exist!')
            return

        # we must look for the adjacent AdjacentVertex (neighbour)  whose vertex is end, and then remove it
        exist = False
        for adj in self._vertices[start]:
            if adj.vertex == end:
                exist = True
                self._vertices[start].remove(adj)
        if not self._directed:
            # we must also look for the AdjacentVertex (neighbour)  whose vertex is end, and then remove it
            for adj in self._vertices[end]:
                if adj.vertex == start:
                    self._vertices[end].remove(adj)

        if not exist: 
            print("({},{}) does not exist!!!!".format(start, end))


    def __str__(self) -> str:
        """ returns a string containing the graph"""
        result = ''
        for v in self._vertices:
            result += '\n'+str(v)+':'
            for adj in self._vertices[v]:
                result += str(adj)+"  "
        result += '\n'
        return result

    def get_adjacent_vertices(self, start: object) -> list:
        """ returns a Python list containing the adjacent
        vertices of vertex. The list only contains the vertices"""
        if start not in self._vertices.keys():
            print(start, ' does not exist!')
            return None
        
        result = []
        for adj in self._vertices[start]:
            result.append(adj.vertex)
        return result


    def get_origins(self, end: object) -> list:
        """ returns a Python list containing those vertices that have
        an edge to vertex. The list is formed with objects of AdjacentVertex"""
        if end not in self._vertices.keys():
            print(end, ' does not exist!')
            return None
        
        result = []
        for v in self._vertices.keys():
            for adj in self._vertices[v]:
                if adj.vertex == end:
                    result.append(v)

        return result
  
  


Aunque el enunciado no lo diga de forma explícita, el problema debe estar basado en el algoritmo de dijskstra (camino mínimo) explicado durante el curso. La siguiente clase ya contiene ese algoritmo:


In [14]:
import math

class Graph2(Graph):

    def minimum_distance(self, distances: dict, visited: dict) -> int:
        """returns the non-visited vertex with the minimum distance"""
        min_vertex = None
        min_dis = math.inf
        for v in self._vertices.keys():
            if not visited[v] and distances[v]< min_dis:
                min_vertex = v
                min_dis = distances[v]
        return min_vertex

    def dijkstra(self, origin: object) -> (list, list):
        """This method calculates the mininum path from origin to the rest of the vertices in the graph. 
        It returns two dictionaries:
        - distances: for each vertex, v, it contains the minimum distance (sum of weights of the minimum path from origin to v). If the graph is not weighted, the sum is the number 
        of edges in the shortest path from origin to v.
        - previous: for each vertex, v, it contains the previous vertex in the path from origin to v"""

        # we must create the two dictionaries
        distances = dict.fromkeys(self._vertices.keys(), math.inf)  # for each vertex (key), the value is minimum distance in the minimum path from origin. At the beginning, all values are infinite
        previous = dict.fromkeys(self._vertices.keys(), None)   # for each vertex (key), the value is the previous node in the minimum path from origin. At the beginning, all values are None. 

        # we also create a dictionary to know what vertices have been already visited. If a vertices has been already visited, we must no visit it againg.
        visited = dict.fromkeys(self._vertices.keys(), False)    # for each vertex (key), the value is a boolean indicating if the vertex has been visited


        # the distance from origin to origin is 0
        distances[origin] = 0

        # the loop will be executed as many times as the number of vertices
        # In each iteration, we will visite the non-visited vertex with th minimum distance
        for _ in range(len(self._vertices.keys())):
            # obtains the non-visited vertex with the minimum distance
            u = self.minimum_distance(distances, visited)

            # we must visit it 
            visited[u] = True
            # we must visit its adjacent vertices to update the distance 
            for adj in self._vertices[u]:
                v = adj.vertex
                w = adj.weight
                # if the adjacent vertex has not been visited and its accumulated distance is greater than 
                # the distance from u, we must update its distance and its previous vertex
                if not visited[v] and distances[v] > distances[u] + w:
                    # As the distance from u and the weight is lower, we must update the accumulated distance in v
                    distances[v] = distances[u] + w
                    # we must also update the previous vertex in the shortes path
                    previous[v] = u

        # we returns the two dictionaries
        return distances, previous

    def minimum_path(self, start: object, end: object) -> (list, int): 
        """returns a list with the minimum path from start to origin. It also returns the distance"""

        # first, we check that the vertices exist in the graph.
        # If some of them does not exist, we return None
        if start not in self._vertices.keys():
            print(start, " is not a vertex")
            return None, -1

        if end not in self._vertices.keys():
            print(end, " is not a vertex")
            return None, -1

        # we use dijkstra to obtain all the minimum paths from start
        distances, previous = self.dijkstra(start)

        # we only want the minimum path from end

        if distances[end] == math.inf: # This means that end is not reachable from start
            print("there is no path from ", start, " to ", end)
            # we return the empty list and infinite
            return [], math.inf

        # result is the list that will contain the minimum path from start to end.
        # We can use previous to re-build the path
        result = [end]
        prev = previous[end]
        while prev != None:
            result.append(prev)
            prev = previous[prev]
        # we use reverse to reverse the list and obtain the path from start to end
        result.reverse()
        # we return the list and also the distance
        return result, distances[end]
        

                

<img src='https://infinitegraph.com/wp-content/uploads/2021/04/WeightedGraph01.png' width='400'>


In [15]:
labels = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'Z']
g = Graph2(labels, False)

g.add_edge('A','B', 4)
g.add_edge('A','D', 3)
g.add_edge('A','E', 4)

g.add_edge('B','C', 5)

g.add_edge('C','D', 6)
g.add_edge('C','Z', 10)

g.add_edge('D','E', 2)
g.add_edge('D','Z', 8)

g.add_edge('E','F', 4)

g.add_edge('F','G', 3)

g.add_edge('G','Z', 8)

print(g)

for start in g._vertices.keys():
    for end in g._vertices.keys():
        path, d = g.minimum_path(start, end)
        print("minimum_path from {} to {}: {}, distance: {}".format(start, end, path, d))
    print()
        



A:(B,4)  (D,3)  (E,4)  
B:(A,4)  (C,5)  
C:(B,5)  (D,6)  (Z,10)  
D:(A,3)  (C,6)  (E,2)  (Z,8)  
E:(A,4)  (D,2)  (F,4)  
F:(E,4)  (G,3)  
G:(F,3)  (Z,8)  
Z:(C,10)  (D,8)  (G,8)  

minimum_path from A to A: ['A'], distance: 0
minimum_path from A to B: ['A', 'B'], distance: 4
minimum_path from A to C: ['A', 'D', 'C'], distance: 9
minimum_path from A to D: ['A', 'D'], distance: 3
minimum_path from A to E: ['A', 'E'], distance: 4
minimum_path from A to F: ['A', 'E', 'F'], distance: 8
minimum_path from A to G: ['A', 'E', 'F', 'G'], distance: 11
minimum_path from A to Z: ['A', 'D', 'Z'], distance: 11

minimum_path from B to A: ['B', 'A'], distance: 4
minimum_path from B to B: ['B'], distance: 0
minimum_path from B to C: ['B', 'C'], distance: 5
minimum_path from B to D: ['B', 'A', 'D'], distance: 7
minimum_path from B to E: ['B', 'A', 'E'], distance: 8
minimum_path from B to F: ['B', 'A', 'E', 'F'], distance: 12
minimum_path from B to G: ['B', 'A', 'E', 'F', 'G'], distance: 15
minimum_path 

# Problema 5:

Dado un un grafo no ponderado. Implementa la función
minimum_path, que reciba dos vértices, start y end, y devuelve una lista de
Python con los vértices que forman el camino mínimo desde start a end, ambos
inclusive. La función minimum_path debe utilizar el algoritmo de camino mínimo
de Dijkstra. Como el grafo es un grafo no ponderado, se considera como
camino mínimo aquel que tenga un menor número de aristas. Está permitido
utilizar estructuras de Python como las listas o los diccionarios.

## Solución: 

En realidad, la misma implementación anterior sirve para un grafo no ponderado ya que el peso de las aristas será 1. Entonces la distancia del camino mínimo de un vértice a otro será igual al número de aristas que hay en dicho camino. 



In [16]:
labels = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'Z']
g = Graph2(labels, False)

g.add_edge('A','B')
g.add_edge('A','D')
g.add_edge('A','E')

g.add_edge('B','C')

g.add_edge('C','D')
g.add_edge('C','Z')

g.add_edge('D','E')
g.add_edge('D','Z')

g.add_edge('E','F')

g.add_edge('F','G')

g.add_edge('G','Z')

print(g)

for start in g._vertices.keys():
    for end in g._vertices.keys():
        path, d = g.minimum_path(start, end)
        print("minimum_path from {} to {}: {}, distance: {}".format(start, end, path, d))
    print()


A:(B,1)  (D,1)  (E,1)  
B:(A,1)  (C,1)  
C:(B,1)  (D,1)  (Z,1)  
D:(A,1)  (C,1)  (E,1)  (Z,1)  
E:(A,1)  (D,1)  (F,1)  
F:(E,1)  (G,1)  
G:(F,1)  (Z,1)  
Z:(C,1)  (D,1)  (G,1)  

minimum_path from A to A: ['A'], distance: 0
minimum_path from A to B: ['A', 'B'], distance: 1
minimum_path from A to C: ['A', 'B', 'C'], distance: 2
minimum_path from A to D: ['A', 'D'], distance: 1
minimum_path from A to E: ['A', 'E'], distance: 1
minimum_path from A to F: ['A', 'E', 'F'], distance: 2
minimum_path from A to G: ['A', 'E', 'F', 'G'], distance: 3
minimum_path from A to Z: ['A', 'D', 'Z'], distance: 2

minimum_path from B to A: ['B', 'A'], distance: 1
minimum_path from B to B: ['B'], distance: 0
minimum_path from B to C: ['B', 'C'], distance: 1
minimum_path from B to D: ['B', 'A', 'D'], distance: 2
minimum_path from B to E: ['B', 'A', 'E'], distance: 2
minimum_path from B to F: ['B', 'A', 'E', 'F'], distance: 3
minimum_path from B to G: ['B', 'C', 'Z', 'G'], distance: 3
minimum_path from B to Z